In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import powerlaw
import seaborn as sns
import numpy as np
import geopy.distance
import geopandas as gpd

In [ ]:
# Load OpenFlights airport data from the CSV file
OF_airports = pd.read_csv(
    "data/airports.dat",  # Path to the airports data file
    header=None,  # Indicates there are no headers in the file
    names=["Airport ID", "Name", "City", "Country", "IATA/FAA",
           "ICAO", "Latitude", "Longitude", "Altitude", "Timezone", "DST",
           "Tz database timezone", "Type", "Source"],  # Custom column names in the CSV of airports
    sep=","  # Columns are separated by commas
)

# Load OpenFlights routes data from the CSV file
OF_routes = pd.read_csv(
    "data/routes.dat",  # Path to the routes data file
    header=None,  # Indicates there are no headers in the file
    names=["Airline", "Airline ID", "Source airport", "Source airport ID",
           "Destination airport", "Destination airport ID",
           "Codeshare", "Stops", "Equipment"]  # Custom column names in the CSV of the routes
)

# Filter and clean OpenFlights data
OF_routes = OF_routes[OF_routes['Codeshare'].notnull()]  # Keep only rows with non-null codeshare values
OF_routes = OF_routes[OF_routes['Source airport ID'] != "\\N"]  # Exclude rows where Source airport ID is missing
OF_routes = OF_routes[OF_routes['Destination airport ID'] != "\\N"]  # Exclude rows where Destination airport ID is missing
OF_routes = OF_routes[OF_routes['Source airport ID'] != OF_routes['Destination airport ID']]  # Exclude self-loops (routes where source = destination)

# Create weighted OpenFlights network
# Based on the route we weighted it
OF_routes = OF_routes.groupby(['Source airport ID', 'Destination airport ID']) \
    .size() \
    .reset_index(name='w')  # Count the number of routes between each pair of airports and assign as weight
OF_routes.columns = ['i', 'j', 'w']  # Rename columns for clarity
OF_routes['i'] = OF_routes['i'].astype(int)  # Ensure the source airport IDs are integers
OF_routes['j'] = OF_routes['j'].astype(int)  # Ensure the destination airport IDs are integers
OF_routes['w'] = OF_routes['w'].astype(int)  # Ensure the weights are integers

# Symmetrize the network by summing the weights of bidirectional edges
OF_routes = (
    pd.concat([
        OF_routes,  # Original data
        OF_routes.rename(columns={'i': 'j', 'j': 'i'})  # Swap source and destination columns
    ])
    .groupby(['i', 'j'], as_index=False)
    .agg({'w': 'sum'})  # Sum weights of bidirectional edges
)

# Keep only one direction of the edge (i < j)
OF_routes = OF_routes[OF_routes['i'] < OF_routes['j']]

# Filter OF_routes to keep only the routes with airports listed in OF_airports
OF_routes = OF_routes[OF_routes['i'].isin(OF_airports['Airport ID'])]  # Keep edges where the source airport exists in the airport dataset
OF_routes = OF_routes[OF_routes['j'].isin(OF_airports['Airport ID'])]  # Keep edges where the destination airport exists in the airport dataset

# Build the OpenFlights graph
OF_graph = nx.Graph()  # Initialize an undirected graph
for _, row in OF_routes.iterrows():
    source_cords = (OF_airports.loc[OF_airports['Airport ID'] == row['i'], 'Latitude'].values[0],
                    OF_airports.loc[OF_airports['Airport ID'] == row['i'], 'Longitude'].values[0])
    dest_cords = (OF_airports.loc[OF_airports['Airport ID'] == row['j'], 'Latitude'].values[0],
                    OF_airports.loc[OF_airports['Airport ID'] == row['j'], 'Longitude'].values[0])
    distance = geopy.distance.geodesic( source_cords, dest_cords).km
    OF_graph.add_edge(int(row['i']), int(row['j']), weight=row['w'], distance=distance)  # Add edges with weights to the graph


In [ ]:
# # Filter the graph to keep only the largest connected component
OF_graph = OF_graph.subgraph(max(nx.connected_components(OF_graph), key=len))

# # Print the number of nodes in the largest connected component
# print(f'Number of nodes in the largest connected component: {len(list_of_component[0])}')
# # Keep only the largest connected component
# OF_graph = OF_graph.subgraph(list_of_component[0])

# Print the number of nodes and edges
print(f'Number of nodes: {OF_graph.number_of_nodes()}')
print(f'Number of edges: {OF_graph.number_of_edges()}')


In [ ]:
# create a dictionary in which the keys are the airport name and the values are the value of the airport in df
airport_dict = OF_airports.set_index('Airport ID').T.to_dict()

# set the attribute for each node in the graph
nx.set_node_attributes(OF_graph, airport_dict)

In [ ]:
# remove low-degree nodes
G = OF_graph.copy()
low_degree = [n for n, d in G.degree() if d < 10]
G.remove_nodes_from(low_degree)
labels = nx.get_node_attributes(G, 'City')

# compute centrality
centrality = nx.betweenness_centrality(G, weight='distance')
# compute community structure
lpc = nx.community.label_propagation_communities(G)
community_index = {n: i for i, com in enumerate(lpc) for n in com}

#### draw graph ####
fig, ax = plt.subplots(figsize=(30, 15))
pos = nx.spring_layout(G, k=11, seed=4572321, weight='weight', iterations=1000, scale=15)
node_color = [community_index[n] for n in G]
node_size = [v * 20000 for v in centrality.values()]
nx.draw_networkx(
    G,
    pos=pos,
    with_labels=True,
    # with_labels=False,
    labels=labels,
    node_color=node_color,
    node_size=node_size,
    edge_color="gainsboro",
    alpha=0.5,
)

# Resize figure for label readability
ax.margins(0.1, 0.05)
fig.tight_layout()
plt.axis("off")
plt.show()

In [ ]:
labels = nx.get_node_attributes(OF_graph, 'Name')
fig = plt.figure(figsize=(120, 80))
# Plot the graph with more space between nodes
# nx.draw_networkx(OF_graph, node_size=10, edge_color='gray', alpha=0.5, with_labels=True, labels=labels)
nx.draw_networkx(OF_graph, node_size=10, edge_color='gray', alpha=0.5, with_labels=False)
plt.title('Airport and flight network', fontsize=15)
plt.savefig('images/networkx_graph.png')
plt.show()

In [ ]:
for node in OF_graph.nodes():
    x = airport_dict[node]['Longitude']
    y = airport_dict[node]['Latitude']
    pos = (x, y)
    OF_graph.nodes[node]['pos'] = pos

# Ottieni le posizioni dei nodi
pos = nx.get_node_attributes(OF_graph, 'pos')
labels = nx.get_node_attributes(OF_graph, 'City')
bbox = dict(boxstyle="round", ec="white", fc="white", alpha=0.3)

# Disegna il grafo
#plt.figure(figsize=(100, 50))
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world: gpd.GeoDataFrame = gpd.read_file(url)
world.plot(figsize=(90,40), edgecolor='gray', color='gray', alpha=0.5)

nx.draw_networkx_nodes(OF_graph, pos,node_color='red', node_size=90, margins=0)
nx.draw_networkx_edges(OF_graph, pos, alpha=0.1)
nx.draw_networkx_labels(OF_graph, pos, labels=labels, font_size=10, font_color='black', bbox=bbox)
plt.title('Airport and flight network', fontsize=15)
plt.tight_layout()
plt.savefig('images/networkx_draw_coordinate_graph.png')
plt.show()

In [ ]:
import folium

m = folium.Map(location=[41.8719, 12.5674], zoom_start=6, zoom_control=True, scrollWheelZoom=False)
folium.TileLayer('cartodbpositron').add_to(m)
for k, pos in nx.get_node_attributes(OF_graph, 'pos').items(): #posizionamento nodi del grafo sulla mappa
    tooltip = f"{OF_graph.nodes[k]['Name']} ({k})"
    folium.CircleMarker(
        location=[pos[1], pos[0]], #restituisce posizione nodo (lat e long)
        radius=1,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        tooltip=tooltip
    ).add_to(m)

# add weighted edges
for edge in OF_graph.edges(): # per ogni arco prende i due nodi collegati
    node1 = edge[0]
    node2 = edge[1]
    try:
        pos1 = OF_graph.nodes[node1]['pos'][::-1] #[::-1]da formato xy a lat long
        pos2 = OF_graph.nodes[node2]['pos'][::-1]
    except KeyError:
        continue
    #tooltip = f"{OF_graph.nodes[node1]["Name"]} - {OF_graph.nodes[node2]["Name"]} ({OF_graph.edges[node1, node2]['weight']})"
    #tooltip scrive Aeroporto partenza (nodo1) - Aeroporto arrivo (nodo2) (Peso dell'arco)
    tooltip = f"{OF_graph.nodes[node1]['Name']} - {OF_graph.nodes[node2]['Name']} ({OF_graph.edges[node1, node2]['weight']})"
    weight = OF_graph.edges[node1, node2]['weight'] / 2
    #divide per due (avanti ed indietro --> monodirezionale)

    folium.PolyLine([pos1, pos2], color="blue", weight=weight, opacity=0.2, tooltip=tooltip).add_to(m)
m

In [ ]:
degree = dict(nx.degree(OF_graph))
# print("Degree:", degree)

# Calculate the degree centrality of the nodes
degree_centrality = nx.degree_centrality(OF_graph)
# print("Degree Centrality:", degree_centrality)

betweenness_centrality = nx.betweenness_centrality(OF_graph, weight="weight")
# print("Betweenness Centrality:", betweenness_centrality)

eigenvector_centrality = nx.eigenvector_centrality(OF_graph, max_iter=1000, weight="weight")
# print("Eigenvector Centrality:", eigenvector_centrality)

closeness_centrality = nx.closeness_centrality(OF_graph, distance="distance")
# print("Closeness Centrality:", closeness_centrality)

clustering_coefficient = nx.clustering(OF_graph, weight="weight")
# print("Clustering Coefficient:", clustering_coefficient)

pagerank = nx.pagerank(OF_graph, weight="weight")
# print("Pagerank:", pagerank)

core_number = nx.core_number(OF_graph)
# print("Core Numbers:", core_number)

In [ ]:
density = nx.density(OF_graph) #densità del grafo
print("Density:", density)

if nx.is_connected(OF_graph): #se grafo connesso
    avg_path_length = nx.average_shortest_path_length(OF_graph, weight='distance') #lunghezza media del percorso
    diameter = nx.diameter(OF_graph) #diametro = massima distanza dei nodi nel grafo
    print(f"Average path length: {avg_path_length}")
    print(f"Diameter of network: {diameter}")
else:
    avg_path_length = None
    diameter = None

connected_components = list(nx.connected_components(OF_graph)) #lista dei componenti connessi --> sottografi dei grafi
connectedness = len(connected_components) #numero dei componenti trovati
print("Number of Connected Components:", connectedness)

#COEFFICIENTE DI ASSORTIVITA'
#nodi dello stesso livello a collegarsi tra loro: (
    #.>0 --> nodi con gradi simili si collegano, .<0 --> nodi con gradi diversi si collegano)
    #.=0 --> nodi dove non c'è correlazione
assortativity = nx.degree_assortativity_coefficient(OF_graph)
print("Assortativity:", assortativity)

#bridges --> arco critico
bridges = list(nx.bridges(OF_graph))
# print("Bridges:", bridges)
print("Number of bridges", len(bridges))

In [ ]:
# create a dataframe with the metrics for each node
df = pd.DataFrame([degree, degree_centrality, betweenness_centrality, eigenvector_centrality, closeness_centrality, clustering_coefficient, pagerank, core_number]).T

# rename the columns
df.columns = ['Degree', 'Degree Centrality', 'Betweenness Centrality', 'Eigenvector Centrality', 'Closeness Centrality', 'Clustering Coefficient', 'Pagerank', 'Core Number']

# merge the dataframe with the airport data
df = OF_airports.merge(df, right_index=True, left_on='Airport ID')
df

# Identification of the main hubs

- Degree Centrality
    - It measures the number of direct connections (routes) an airport has.
    - High degree centrality identifies airports with the most direct routes and is a good first approximation of hub importance.
- Betweenness Centrality
    - It measures how often an airport lies on the shortest paths between other airports.
    - High betweenness centrality highlights airports that act as intermediaries or key transit points.
- Closeness Centrality
    - It measures the average shortest path distance from an airport to all others.
    - High closeness centrality identifies airports that can quickly reach all others, making them strategically located.
- Eigenvector Centrality
    - It measures the importance of an airport based on its connections to other important airports.
    - Identifies influential hubs connected to other major hubs.
- Pagerank
    - It measures the importance of an airport based on the importance of its connections.
    - Highlights influential nodes even if their connections are to less significant nodes.


In [ ]:
n_best = 10 #prende i primi 10 valori degli aeroporti per ogni metrica delle colonne

columns = [
    'Degree', 'Degree Centrality', 'Betweenness Centrality', 'Eigenvector Centrality',
    'Closeness Centrality', 'Clustering Coefficient', 'Pagerank', 'Core Number'
]
titles = [
    f'Top {n_best} airports with the highest degree',
    f'Top {n_best} airports with the highest degree centrality',
    f'Top {n_best} airports with the highest betweenness centrality',
    f'Top {n_best} airports with the highest eigenvector centrality',
    f'Top {n_best} airports with the highest closeness centrality',
    f'Top {n_best} airports with the highest clustering coefficient',
    f'Top {n_best} airports with the highest pagerank',
    f'Top {n_best} airports with the highest core number'
]

# Imposta il numero di subplot
fig, axes = plt.subplots(2, 4, figsize=(20, 10))  # 2 righe, 4 colonne

# Loop attraverso le colonne e i titoli
for ax, col, title in zip(axes.flatten(), columns, titles):
    ax: plt.Axes
    data = df.sort_values(by=col, ascending=False).head(n_best)
    sns.barplot(data=df.sort_values(by=col, ascending=False).head(n_best), x='Name', y=col, ax=ax)
    ax.set_title(title)
    ax.axes.xaxis.set_ticks(ax.get_xticks())
    ax.set_xticklabels([f"{city} ({iata})" for city, iata in zip(data['City'], data['IATA/FAA'])], rotation=80)

# Aggiungi spaziatura tra i subplot
plt.tight_layout()
plt.show()

In [ ]:
titles = [ #distrubuzioni delle metrice
    'Degree distribution', 'Degree Centrality distribution', 'Betweenness Centrality distribution',
    'Eigenvector Centrality distribution', 'Closeness Centrality distribution', 'Clustering Coefficient distribution',
    'Pagerank distribution', 'Core Number distribution'
]

titles_linear = [f"{t} in linear scale" for t in titles]

# Configura la figura con 2x4 subplot nella schermata
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# Loop su colonne e titoli per creare gli istogrammi
for ax, col, title in zip(axes.flatten()[:len(columns)], columns, titles_linear): #associazione dei subplot alla metrica corrispondente e titolo
    sns.histplot(df[col], ax=ax, stat="density", bins=20)
    ax.set_title(title)
    #df[col] -->prende i dati del dataframe per ogni colonna corrispondente
    #stat="density" --> istogramma come densità di probabilità normalizzato a 1
    #bins = 20 -->  numero di intervalli (20)

# Rimuovi eventuali assi inutilizzati nell'ultimo subplot
if len(columns) < len(axes.flatten()):
    for ax in axes.flatten()[len(columns):]:
        ax.axis('off')

# Adatta layout finale
plt.tight_layout()
plt.show()


In [ ]:
# Configura la figura con 2 righe e 4 colonne
fig, axes = plt.subplots(2, 4, figsize=(20, 10))

titles_exp = [f"{t} in log-log scale" for t in titles]

# Loop su colonne e titoli per creare i grafici
for ax, col, title in zip(axes.flatten(), columns, titles_exp):
    sns.histplot(df[col], ax=ax, stat="density", bins=20, log_scale=True)
    ax.set_yscale("log")
    # plot the distribution of the metrics with a log scale. in the vertical axis is visualized the fraction of nodes with that value
    ax.set_ylim(0.0001, 5)
    ax.set_title(title)

# Adatta layout finale
plt.tight_layout()
plt.show()

In [ ]:
# powerlaw calculation
fit: powerlaw.Fit = powerlaw.Fit(list(degree.values()), discrete=True)
#adatta distribuzione dei gradi della rete ad una distribuzione di potenza
#list(degree.values()) --> prende valori dei gradi dei nodi di un grafo e li converte in lista
# discrete = True --> dati forniti sono discreti, cioè numeri interi come i grafi
# self.alpha = 1 + (self.n / sum(log(data / (self.xmin - .5))))

#Fase di fitting per capire se dati seguono una distribuzione o no
#Stima di alpha --> parametro che caratterizza pendenza della distribuzione
alpha = fit.power_law.alpha
print(f'alpha has value: {alpha}')


plt.figure(figsize=(10, 5))
fit.plot_pdf(label='Probability Density Function (PDF)')
plt.legend()
plt.show()


In [ ]:
# Examine if highly connected airports have low clustering (common for hubs)
fig, ax = plt.subplots(figsize=(10, 6))
sns.scatterplot(data=df, ax=ax, x='Degree', y='Clustering Coefficient')
ax.set_yscale('log')
ax.set_xscale('log')
plt.xlabel('Degree')
plt.ylabel('Clustering Coefficient')
plt.title('Clustering Coefficient vs Degree')
plt.show()

# Identification of regional clusters

In [ ]:
# Compute the best partition using the Louvain method
# partition = community_louvain.best_partition(OF_graph)
clusters_list = nx.community.louvain_communities(OF_graph, weight='weight')
clusters = {}
for i, cluster in enumerate(clusters_list):
    clusters[i] = cluster

partition = {}
for cluster_id, airport_ids in clusters.items():
    for airport_id in airport_ids:
        partition[airport_id] = cluster_id


# Assign community as a node attribute
nx.set_node_attributes(OF_graph, partition, 'community')

# Get the position for nodes
pos = nx.get_node_attributes(OF_graph, 'pos')

# Generate a color map for different communities
# cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
cmap = plt.get_cmap("viridis", 7)


# Plot the graph, coloring nodes based on their community
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
world: gpd.GeoDataFrame = gpd.read_file(url)
world.plot(figsize=(90,40), edgecolor='gray', alpha=0.5, color='gray')
nx.draw_networkx_nodes(OF_graph, pos, partition.keys(), node_size=90, cmap=cmap, node_color=list(partition.values()), margins=0)
nx.draw_networkx_edges(OF_graph, pos, alpha=0.1)
plt.title('Regional clusters (communities) in the airport network', fontsize=50)
plt.tight_layout()
plt.savefig('images/community_clustering_graph.png')
# plt.show()

# Stampa per ogni cluster gli ID e i nomi degli aeroporti appartenenti
for cluster_id, airport_ids in clusters.items():
    airport_names = [airport_dict[airport_id]['City'] for airport_id in airport_ids]
    print(f"Cluster {cluster_id}: {airport_names}")

# Optionally, print out the community assignment for each node
#for airport_id, community_id in partition.items():
    #print(f"Airport ID {airport_id} is in community {community_id}")

# Robustness simulation

## Calcolo della Connettività

In [ ]:
initial_connectivity = nx.node_connectivity(OF_graph)
print(f'Initial connectivity: {initial_connectivity}')

# Rimozione degli hub e ricalcolo della connettività
hubs = df.sort_values(by='Degree', ascending=False).head(10)['Airport ID']
OF_graph_removed_hubs = OF_graph.copy()
OF_graph_removed_hubs.remove_nodes_from(hubs)
connectivity_after_hub_removal = nx.node_connectivity(OF_graph_removed_hubs)
print(f'Connectivity after removing hubs: {connectivity_after_hub_removal}')


## Calcolo della Frammentazione

In [ ]:
largest_cc = max(nx.connected_components(OF_graph), key=len)
fragmentation = 1 - (len(largest_cc) / OF_graph.number_of_nodes())
print(f'Fragmentation before removing hubs: {fragmentation}')

largest_cc = max(nx.connected_components(OF_graph_removed_hubs), key=len)
fragmentation = 1 - (len(largest_cc) / OF_graph_removed_hubs.number_of_nodes())
print(f'Fragmentation after removing hubs: {fragmentation}')

## Calcolo della Compattezza

In [ ]:
if nx.is_connected(OF_graph):
    initial_compactness = 1 / nx.average_shortest_path_length(OF_graph)
    print(f'Initial compactness: {initial_compactness}')

if nx.is_connected(OF_graph_removed_hubs):
    compactness_after_hub_removal = 1 / nx.average_shortest_path_length(OF_graph_removed_hubs)
    print(f'Compactness after removing hubs: {compactness_after_hub_removal}')
else:
    print("The graph is no longer connected after removing hubs.")
    component_number = len([c for c in nx.connected_components(OF_graph_removed_hubs)])
    print(f"Now the graph has {component_number} component")


## Analisi della Centralità

In [ ]:
betweenness_before = nx.betweenness_centrality(OF_graph)
betweenness_after = nx.betweenness_centrality(OF_graph_removed_hubs)

# Analizza la variazione nella betweenness centrality
changes = {
    node: betweenness_after[node] - betweenness_before[node]
    for node in betweenness_before.keys() & betweenness_after.keys()
}

biggest_changes = sorted(changes.items(), key=lambda x: x[1], reverse=True)[:10]
print(biggest_changes)

# filter df with the biggest changes
df_biggest_changes = df[df['Airport ID'].isin([k for k, v in biggest_changes])]
df_biggest_changes

In [ ]:
sorted(betweenness_after.items(), key=lambda x: x[1], reverse=True)[:10]